In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
#import minmaxx scaler
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler, StandardScaler,MaxAbsScaler,PowerTransformer,QuantileTransformer
import statsmodels.api as sm
from get_model import get_model
import torch

/home/cynthia/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def prepare_dataset(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['filled']=df['meter_reading'].apply(lambda x: 0)
    existing_hours = df['timestamp'].dt.floor('H').unique()

    start_date = df['timestamp'].min().replace(minute=0, second=0)
    end_date = df['timestamp'].max().replace(minute=0, second=0)
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    all_hours_present = all(hour in existing_hours for hour in date_range)
    if not(all_hours_present):
        complete_df = pd.DataFrame({'timestamp': date_range})
        df = complete_df.merge(df, on='timestamp', how='left')
        df['filled']=df['filled'].fillna(1)
        df['meter_reading'] = df['meter_reading'].interpolate(method='linear', limit_direction='both')
        df.reset_index(inplace=True, drop=True)
    #interpolate the readings
    #df['meter_reading'] = df['meter_reading'].interpolate(method='linear', limit_direction='both')
    
    #apply minmax scaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    df['meter_reading'] = scaler.fit_transform(df['meter_reading'].values.reshape(-1,1))
    grouped_df = df.groupby(df['timestamp'].dt.date)

    # Step 3: Aggregate 'meter_reading' values into a list for each day
    aggregated_df = grouped_df.agg({'meter_reading': list, 'anomaly': list, 'filled':list}).reset_index()

    # Step 4: Rename columns and sort by date
    aggregated_df.columns = ['date', 'readings', 'anomalies','filled']
    aggregated_df = aggregated_df.sort_values(by='date')

    # Display the aggregated dataframe
    aggregated_df["length"] = aggregated_df["readings"].apply(lambda lst: len([x for x in lst if not pd.isna(x)]))
    aggregated_df["no_anomalies"] = aggregated_df["anomalies"].apply(lambda x: True if all(val == 0 for val in x) else False)
    #aggregated_df.to_csv("LEAD/buildings/building_{}.csv".format(i),index=False)
    df=aggregated_df[aggregated_df["length"]==24]
    df['cycle'] = df["readings"].apply(lambda x: sm.tsa.filters.hpfilter(x, 2)[0])
    df['trend'] = df["readings"].apply(lambda x: sm.tsa.filters.hpfilter(x, 2)[1])
    df["months"] = df["date"].apply(lambda x: str(x.month))
    df["weekday"] = df["date"].apply(lambda x: str(x.weekday()))
    df["weekend"] = df["weekday"].apply(lambda x: 1 if x in ["5","6"] else 0)
    return df

def generate_data(df,column):
    data=df[column]
    data=np.concatenate(data)
    #tmp=data[0:24]
    data=data.reshape((1,df.shape[0],24))
    #print(data[0,0,:]==tmp)
    return data

def prepare_line(col1, col2,col3):
    columns=[col1, col2,col3]
    dataset=[]
    for column in columns:
        data=np.array(column)
        data=data.reshape(1,1,24)
        dataset.append(data)
    data_array=np.concatenate(dataset)
    data_array=np.transpose(data_array, (1,0,2))
    #check=np.concatenate([col1,col2,col3])
    #check=check.reshape(1,3,24)
    #print(check==data_array)
    return(data_array)
def prepare_line0(col1, col2):
    columns=[col1, col2]
    dataset=[]
    for column in columns:
        data=np.array(column)
        data=data.reshape(1,1,24)
        dataset.append(data)
    data_array=np.concatenate(dataset)
    data_array=np.transpose(data_array, (1,0,2))
    #check=np.concatenate([col1,col2,col3])
    #check=check.reshape(1,3,24)
    #print(check==data_array)
    return(data_array)

In [3]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
def get_cluster_labels(df,column):
    def get_majority(lst):
        return max(set(lst), key=lst.count)
    X = np.array(df[column].tolist())
    # do PCA with 2 components on X
    pca = PCA(n_components=2)
    X = pca.fit_transform(X)

    kmeans = KMeans(n_clusters=3)
    kmeans.fit(X)
    max_label = get_majority(kmeans.labels_.tolist())
    labels=[0 if i==max_label else 1 for i in kmeans.labels_ ]
    return labels


In [4]:
#files=os.listdir('dataset/train')[0:5]
import io
import sys
from contextlib import redirect_stdout
from tqdm import tqdm
from IPython.display import clear_output
captured_output = io.StringIO()
files=["118.csv","246.csv","1245.csv","1311.csv","1141.csv"]
"""
models=['AE',
 'VAE',
 'BETA',
 'VAE_LinNF',
 'VAE_IAF',
 'DBVAE',
 'IWVAE',
 'MIWAE',
 'CIWAE']
"""
models=['WAE',
'INFOVAE',
 'VAMP',
 'SVAE',
 'PVAE',
 'VQVAE',
 'HVAE',
 'RAE_GP',
 'RHVAE']
pbar=tqdm(total=len(models))
try:
    for model_name in models:
        original_stdout = sys.stdout
        sys.stdout = captured_output
        for file in files:
            
            dataset=pd.read_csv('dataset/train/'+file)
            dataset=prepare_dataset(dataset)
            columns_cluster=["cycle","trend"]
                        
            for column_cluster in columns_cluster:
                dataset['kmeans_{}'.format(column_cluster)]=np.nan
            for month in dataset["months"].unique():
                for day in dataset["weekend"].unique():
                        df=dataset[(dataset["months"]==month) & (dataset["weekend"]==day)].copy()
                        
                        
                        for column_cluster in columns_cluster:
                                
                                df['kmeans_{}'.format(column_cluster)]=get_cluster_labels(df,column_cluster)
                                #create a new column with the cluster labels in dataset and initialize it with NAN
                                
                                
                        dataset.update(df)
            for column_cluster in columns_cluster:
                if not(os.path.exists("experiment_2/csv/{}".format(column_cluster))):
                    os.makedirs("experiment_2/csv/{}".format(column_cluster))
                train=dataset[dataset['kmeans_{}'.format(column_cluster)]==0.0]
                test=dataset[dataset['kmeans_{}'.format(column_cluster)]==1.0]
                for month in dataset["months"].unique():
                    for day in dataset["weekend"].unique():
                        
                        train_data=train[(train["months"]==month) & (train["weekend"]==day)]
                        test_data=test[(test["months"]==month) & (test["weekend"]==day)]
                        train_data.reset_index(inplace=True)
                        test_data.reset_index(inplace=True) 
                        if train_data.shape[0]==0:
                            split=0.5
                            train_data=test_data[0:int(split*test_data.shape[0])]
                            eval_data=test_data[int(split*test_data.shape[0]):]
                        else:
                            if test_data.shape[0]==0:
                                eval_data=None
                            else:
                                eval_data=test_data
                        train_columns=["readings","cycle","trend"] #,"trend"
                        train_numpy=np.concatenate([generate_data(train_data,column) for column in train_columns])
                        train_numpy=train_numpy.transpose((1,0,2))

                        """
                        if test_data.shape[0]!=0:
                            eval_numpy=np.concatenate([generate_data(eval_data,column) for column in train_columns])
                            eval_numpy=eval_numpy.transpose((1,0,2))
                            #print(train_data.shape,train_numpy.shape,eval_data.shape)"""
                        
                        pipeline = get_model(model_name,dim=len(train_columns),train_batch=train_numpy.shape[0])
                        pipeline(
                        train_data=train_numpy# must be torch.Tensor, np.array or torch datasets
                        )
                        print("Model ready!")
                        my_vae_model=pipeline.model
                        train_data["preprocessed"]=train_data[train_columns].apply(lambda row: prepare_line(*row), axis=1)
                        train_data.reset_index(inplace=True, drop=True)
                        train_data["reconstruction"]=train_data["preprocessed"].apply((lambda row: my_vae_model.reconstruct(torch.from_numpy(row.astype(np.float32))).detach().numpy()))

                        if test_data.shape[0]!=0:
                            eval_data["preprocessed"]=eval_data[train_columns].apply(lambda row: prepare_line(*row), axis=1)
                            eval_data["reconstruction"]=eval_data["preprocessed"].apply((lambda row: my_vae_model.reconstruct(torch.from_numpy(row.astype(np.float32))).detach().numpy()))
                            eval_data.reset_index(inplace=True, drop=True)
                        if test_data.shape[0] != 0:
                            dataset_final=pd.concat([train_data,eval_data])
                        else:
                            dataset_final=train_data
                        dataset_final["reconstruction"]=dataset_final["reconstruction"].apply((lambda x: x.reshape((1, len(train_columns), 24))))
                        dataset_final['difference'] = dataset_final['preprocessed'] - dataset_final['reconstruction']

                        for i, column in enumerate(train_columns):
                            dataset_final['difference_norm_{}'.format(column)] = dataset_final['difference'].apply(lambda x: np.linalg.norm(x[:,i,:],axis=(0)))
                        dataset_final.reset_index(inplace=True,drop=True)
                        #dataset_final.to_csv("experiment_1/csv/building_{}_month_{}_weekday_{}.csv".format(file,month,day),index=False)
                        for ind in dataset_final.index:
                            tmp_df=pd.DataFrame()
                            anomalies=dataset_final.loc[ind,"anomalies"]
                            filled=dataset_final.loc[ind,"filled"]
                            #tmp_df["value"]=difference_norm
                            tmp_df["anomalies"]=anomalies
                            tmp_df["filled"]=filled
                            tmp_df.reset_index(inplace=True,drop=True)
                            date=dataset_final.loc[ind,"date"]
                            tmp_df['datetime'] = pd.to_datetime(date) + pd.to_timedelta(tmp_df.index, unit='h')


                            
                            for column in train_columns:
                                #plt.figure()
                                

                                # Plot the first array in blue where the second array has 0
                                #plt.plot(x_values[array_of_ones_zeros == 0], array_of_floats[array_of_ones_zeros == 0], c='b', label='0')
                                tmp_df["difference_norm_{}".format(column)]=dataset_final.loc[ind,"difference_norm_{}".format(column)]
                                
                                # Plot the first array in red where the second array has 1
                                
                                #plt.ylim(0, 1)
                                #plt.plot(tmp_df[tmp_df["anomalies"]==0]["value"], c='b')
                                #plt.plot(tmp_df[tmp_df["anomalies"]==1]["value"], c='r')
                                #plt.plot(tmp_df[tmp_df["filled"]==1]["value"], c='y')
                                #threshold=0.1*tmp_df["value"].mean()+3*tmp_df["value"].std()
                                #threshold_array=np.ones(24)*threshold
                                #create a label array that has 1 when the value is above threshold and 0 otherwise
                                #label_array = np.where(difference_norm > threshold, 1, 0)
                                #tmp_df["label"]=label_array
                                
                                # create a datetime column that takes the date (yyyy-mm-dd) and adds the corresponding hour (00:00:00 - 23:00:00 according to the index)
                                
                                
                                #plt.plot(threshold_array, c='g')
                                #plt.savefig("experiment_1/plots/building_{}_month_{}_weekday_{}_{}_column_{}.png".format(file,month,day,ind,column))
                                #plt.close()
                            if not(os.path.exists("experiment_2/csv/{}/latent_dim_16/{}".format(column_cluster,model_name))):
                                os.makedirs("experiment_2/csv/{}/latent_dim_16/{}".format(column_cluster,model_name))
                            tmp_df.to_csv("experiment_2/csv/{}/latent_dim_16/{}/building_{}_month_{}_weekend_{}_{}_column_{}.csv".format(column_cluster,model_name,file,month,day,ind,column),index=False)
                clear_output(wait=False)    
        sys.stdout = original_stdout
        pbar.update(1)
except Exception as e:
    sys.stdout = original_stdout
    print(e)
    print(model_name,file,month,day,ind,column)
    print(train_data.shape,eval_data.shape)
    pbar.close()
pbar.close()

  0%|          | 0/9 [00:00<?, ?it/s]Preprocessing train data...
Checking train dataset...
Using Base Trainer

! No eval dataset provided ! -> keeping best model on train.

Model passed sanity check !
Ready for training.

Created dummy_output_dir/WAE_MMD_training_2023-09-28_07-44-38. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 50
 - per_device_train_batch_size: 11
 - per_device_eval_batch_size: 1
 - checkpoint saving every: 300
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.91, 0.995)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0.05
)
Scheduler: <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7f1ffd548eb0>

Successfully launched training !

Training of epoch 1/50: 100%|██████████| 1/1 [00:00<00:00, 19.01batch/s]
--------------------------------------------------------------------------
Train loss: 84.7323
----------------------------------

KeyboardInterrupt: 

AE 246.csv 5 0 4 trend

In [ ]:
tmp_df

In [ ]:
dataset_final

In [ ]:
readings_train=train_numpy[:,0,:]

In [ ]:
readings_train

In [ ]:
dataset_final

In [ ]:
!pip install tensorflow==1.5

In [ ]:
import warnings
from functools import partial

import tensorflow.compat.v1 as tf
from tfsnippet.distributions import Normal
from tfsnippet.modules import VAE, Lambda, Module
from tfsnippet.stochastic import validate_n_samples
from tfsnippet.utils import (VarScopeObject,
                             reopen_variable_scope,
                             is_integer)
from tfsnippet.variational import VariationalInference

from .reconstruction import iterative_masked_reconstruct

__all__ = ['Donut']


def softplus_std(inputs, units, epsilon, name):
    return tf.nn.softplus(tf.layers.dense(inputs, units, name=name)) + epsilon


def wrap_params_net(inputs, h_for_dist, mean_layer, std_layer):
    with tf.variable_scope('hidden'):
        h = h_for_dist(inputs)
    return {
        'mean': mean_layer(h),
        'std': std_layer(h),
    }


class Donut(VarScopeObject):
    """
    Class for constructing Donut model.

    This class provides :meth:`get_training_loss` for deriving the
    training loss :class:`tf.Tensor`, and :meth:`get_score` for obtaining
    the reconstruction probability :class:`tf.Tensor`.

    Note:
        :class:`Donut` instances will not build the computation graph
        until :meth:`get_training_loss` or :meth:`get_score` is
        called.  This suggests that a :class:`donut.DonutTrainer` or
        a :class:`donut.DonutPredictor` must have been constructed
        before saving or restoring the model parameters.

    Args:
        h_for_p_x (Module or (tf.Tensor) -> tf.Tensor):
            The hidden network for :math:`p(x|z)`.
        h_for_q_z (Module or (tf.Tensor) -> tf.Tensor):
            The hidden network for :math:`q(z|x)`.
        x_dims (int): The number of `x` dimensions.
        z_dims (int): The number of `z` dimensions.
        std_epsilon (float): The minimum value of std for `x` and `z`.
        name (str): Optional name of this module
            (argument of :class:`tfsnippet.utils.VarScopeObject`).
        scope (str): Optional scope of this module
            (argument of :class:`tfsnippet.utils.VarScopeObject`).
    """
    def __init__(self, h_for_p_x, h_for_q_z, x_dims, z_dims, std_epsilon=1e-4,
                 name=None, scope=None):
        if not is_integer(x_dims) or x_dims <= 0:
            raise ValueError('`x_dims` must be a positive integer')
        if not is_integer(z_dims) or z_dims <= 0:
            raise ValueError('`z_dims` must be a positive integer')

        super(Donut, self).__init__(name=name, scope=scope)
        with reopen_variable_scope(self.variable_scope):
            self._vae = VAE(
                p_z=Normal(mean=tf.zeros([z_dims]), std=tf.ones([z_dims])),
                p_x_given_z=Normal,
                q_z_given_x=Normal,
                h_for_p_x=Lambda(
                    partial(
                        wrap_params_net,
                        h_for_dist=h_for_p_x,
                        mean_layer=partial(
                            tf.layers.dense, units=x_dims, name='x_mean'
                        ),
                        std_layer=partial(
                            softplus_std, units=x_dims, epsilon=std_epsilon,
                            name='x_std'
                        )
                    ),
                    name='p_x_given_z'
                ),
                h_for_q_z=Lambda(
                    partial(
                        wrap_params_net,
                        h_for_dist=h_for_q_z,
                        mean_layer=partial(
                            tf.layers.dense, units=z_dims, name='z_mean'
                        ),
                        std_layer=partial(
                            softplus_std, units=z_dims, epsilon=std_epsilon,
                            name='z_std'
                        )
                    ),
                    name='q_z_given_x'
                )
            )
        self._x_dims = x_dims
        self._z_dims = z_dims

    @property
    def x_dims(self):
        """Get the number of `x` dimensions."""
        return self._x_dims

    @property
    def z_dims(self):
        """Get the number of `z` dimensions."""
        return self._z_dims

    @property
    def vae(self):
        """
        Get the VAE object of this :class:`Donut` model.

        Returns:
            VAE: The VAE object of this model.
        """
        return self._vae

    def get_training_loss(self, x, y, n_z=None):
        """
        Get the training loss for `x` and `y`.

        Args:
            x (tf.Tensor): 2-D `float32` :class:`tf.Tensor`, the windows of
                KPI observations in a mini-batch.
            y (tf.Tensor): 2-D `int32` :class:`tf.Tensor`, the windows of
                ``(label | missing)`` in a mini-batch.
            n_z (int or None): Number of `z` samples to take for each `x`.
                (default :obj:`None`, one sample without explicit sampling
                dimension)

        Returns:
            tf.Tensor: 0-d tensor, the training loss, which can be optimized
                by gradient descent algorithms.
        """
        with tf.name_scope('Donut.training_loss'):
            chain = self.vae.chain(x, n_z=n_z)
            x_log_prob = chain.model['x'].log_prob(group_ndims=0)
            alpha = tf.cast(1 - y, dtype=tf.float32)
            beta = tf.reduce_mean(alpha, axis=-1)
            log_joint = (
                tf.reduce_sum(alpha * x_log_prob, axis=-1) +
                beta * chain.model['z'].log_prob()
            )
            vi = VariationalInference(
                log_joint=log_joint,
                latent_log_probs=chain.vi.latent_log_probs,
                axis=chain.vi.axis
            )
            loss = tf.reduce_mean(vi.training.sgvb())
            return loss

    def get_training_objective(self, *args, **kwargs):  # pragma: no cover
        warnings.warn('`get_training_objective` is deprecated, use '
                      '`get_training_loss` instead.', DeprecationWarning)
        return self.get_training_loss(*args, **kwargs)

    def get_score(self, x, y=None, n_z=None, mcmc_iteration=None,
                  last_point_only=True):
        """
        Get the reconstruction probability for `x` and `y`.

        The larger `reconstruction probability`, the less likely a point
        is anomaly.  You may take the negative of the score, if you want
        something to directly indicate the severity of anomaly.

        Args:
            x (tf.Tensor): 2-D `float32` :class:`tf.Tensor`, the windows of
                KPI observations in a mini-batch.
            y (tf.Tensor): 2-D `int32` :class:`tf.Tensor`, the windows of
                missing point indicators in a mini-batch.
            n_z (int or None): Number of `z` samples to take for each `x`.
                (default :obj:`None`, one sample without explicit sampling
                dimension)
            mcmc_iteration (int or tf.Tensor): Iteration count for MCMC
                missing data imputation. (default :obj:`None`, no iteration)
            last_point_only (bool): Whether to obtain the reconstruction
                probability of only the last point in each window?
                (default :obj:`True`)

        Returns:
            tf.Tensor: The reconstruction probability, with the shape
                ``(len(x) - self.x_dims + 1,)`` if `last_point_only` is
                :obj:`True`, or ``(len(x) - self.x_dims + 1, self.x_dims)``
                if `last_point_only` is :obj:`False`.  This is because the
                first ``self.x_dims - 1`` points are not the last point of
                any window.
        """
        with tf.name_scope('Donut.get_score'):
            # MCMC missing data imputation
            if y is not None and mcmc_iteration:
                x_r = iterative_masked_reconstruct(
                    reconstruct=self.vae.reconstruct,
                    x=x,
                    mask=y,
                    iter_count=mcmc_iteration,
                    back_prop=False,
                )
            else:
                x_r = x

            # get the reconstruction probability
            q_net = self.vae.variational(x=x_r, n_z=n_z)  # notice: x=x_r
            p_net = self.vae.model(z=q_net['z'], x=x, n_z=n_z)  # notice: x=x
            r_prob = p_net['x'].log_prob(group_ndims=0)
            if n_z is not None:
                n_z = validate_n_samples(n_z, 'n_z')
                assert_shape_op = tf.assert_equal(
                    tf.shape(r_prob),
                    tf.stack([n_z, tf.shape(x)[0], self.x_dims]),
                    message='Unexpected shape of reconstruction prob'
                )
                with tf.control_dependencies([assert_shape_op]):
                    r_prob = tf.reduce_mean(r_prob, axis=0)
            if last_point_only:
                r_prob = r_prob[:, -1]
            return r_prob


In [ ]:
import tensorflow as tf

from tensorflow import keras as K
from tfsnippet.modules import Sequential

# We build the entire model within the scope of `model_vs`,
# it should hold exactly all the variables of `model`, including
# the variables created by Keras layers.
with tf.variable_scope('model') as model_vs:
    model = Donut(
        h_for_p_x=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        h_for_q_z=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        x_dims=120,
        z_dims=5,
    )

In [ ]:
from donut import DonutTrainer, DonutPredictor

trainer = DonutTrainer(model=model, model_vs=model_vs)
predictor = DonutPredictor(model)

with tf.Session().as_default():
    trainer.fit(train_values, train_labels, train_missing, mean, std)
    test_score = predictor.get_score(test_values, test_missing)

In [ ]:
from donut import iterative_masked_reconstruct

# Obtain the reconstructed `x`, with MCMC missing data imputation.
# See also:
#   :meth:`donut.Donut.get_score`
#   :func:`donut.iterative_masked_reconstruct`
#   :meth:`tfsnippet.modules.VAE.reconstruct`
input_x = ...  # 2-D `float32` :class:`tf.Tensor`, input `x` windows
input_y = ...  # 2-D `int32` :class:`tf.Tensor`, missing point indicators
               # for the `x` windows
x = model.vae.reconstruct(
    iterative_masked_reconstruct(
        reconstruct=model.vae.reconstruct,
        x=input_x,
        mask=input_y,
        iter_count=mcmc_iteration,
        back_prop=False
    )
)
# `x` is a :class:`tfsnippet.stochastic.StochasticTensor`, from which
# you may derive many useful outputs, for example:
x.tensor  # the `x` samples
x.log_prob(group_ndims=0)  # element-wise log p(x|z) of sampled x
x.distribution.log_prob(input_x)  # the reconstruction probability
x.distribution.mean, x.distribution.std  # mean and std of p(x|z)